In [14]:
import numpy as np 
import pandas as pd 

import matplotlib.patches as patches

import os
import random
import xml.etree.ElementTree as ET
import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset,DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import cv2
from PIL import Image

import sys
import torch.optim as optim

from time import time


In [2]:
labels_dict = {1: 'Eurasian_jay',
 2: 'great_spotted_woodpecker',
 3: 'greenfinch',
 4: 'blue_tit',
 5: 'Carduelis',
 6: 'common_redpoll',
 7: 'great_tit',
 8: 'bullfinch',
 9: 'Eurasian_siskin',
 10: 'Eurasian_tree_sparrow',
 11: 'hawfinch',
 12: 'willow_tit',
 13: 'Fieldfare',
 14: 'Common chaffinch'}



In [15]:
class BirdsDetection:
    """
    Class implements detection with Faster R-CNN mobilenet trained for 14 birds species
    """

    def __init__(self, source, out_file = None,
                       weights_path = '/home/costia/faster_rcnn_mobilenet_10feb.pt',
                       conf_lvl = 0.6,labels_dict = labels_dict):
        """
        source - camera url or device
        weigths_path - path to model weights
        conf_lvl - confidence level of predictions
        labels_dict - dictionary of birds dpecies names
        
        """
        self.source = source
        self.weights_path = weights_path
        self.model = self.load_model()
        self.out_file = out_file if out_file is not None else source.split('.')[0]+"_predicted.avi" 
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.conf_lvl = conf_lvl
        self.transform = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                        ])


    def get_video_from_source(self):
        """
        Creates a new video streaming object to extract video frame by frame to make prediction on.
        :returns: opencv2 video capture object
        """
        return cv2.VideoCapture(self.source)

    def load_model(self):
        """
        load Faster R-CNN without weights pytorch hub and  
        load trained weights.
               
        :returns: Pytorch model with weights.
        """
        #
        model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(pretrained=True)
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        num_classes = 14+1 #(n_classes + background)
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
       #load weights
        
        model.load_state_dict(torch.load(self.weights_path))
        model.eval()
        return model

    def score_frame(self, frame):
        """
        Takes a single frame as input, and scores the frame .
        :param frame: input frame in numpy/list/tuple format.
        :returns: Labels and  boxes of objects detected by model in the frame.
        """
        self.model.to(self.device)
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        inputs = self.transform(frame)
        
        inputs = (inputs) #for img in frame]
        #plt.imshow(inputs)
        inputs.unsqueeze_(0)
        #print(inputs.shape)
        inputs = inputs.to(self.device) #for img in inputs]
        with torch.no_grad():
            preds = self.model(inputs)
        torch.cuda.empty_cache()
        
        return preds

    def class_to_label(self, label):
        """
        For a given label value, return bird species name.
        :param x: numeric label
        :return: corresponding bird species
        """
        return labels_dict [label]

    def plot_boxes(self, preds, frame):
        """
        Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
        :param results: contains labels and coordinates predicted by model on the given frame.
        :param frame: Frame which has been scored.
        :returns: Frame with bounding boxes and labels ploted on it.
        """
        
        
        mask = preds[0]['scores'] >= self.conf_lvl
        print('Founded:',preds[0]['scores'])
        confidences = preds[0]['scores'][mask]
        boxes = preds[0]['boxes'][mask]
        labels = preds[0]['labels'][mask]
        
        flag = True
        
        for i, box in enumerate(boxes):
            x_min, y_min, x_max, y_max = map(int,box)
            bgr = (0, 255, 0)
            #print((x_min, y_min), (x_max, y_max))
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), bgr, 2)
            cv2.putText(frame, self.class_to_label(int(labels[i])), (x_min, y_min), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)
        return frame
      

    def __call__(self):
        """
        This function is called when class is executed, it runs the loop to read the video frame by frame,
        and write the output into a new file.
        :returns: void
        """
        player = self.get_video_from_source()
        assert player.isOpened()
        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        #four_cc = cv2.VideoWriter_fourcc(*"MJPG")
        #out = cv2.VideoWriter(self.out_file, four_cc, 30, (x_shape, y_shape))
        i = 0
        while True:
            
            start_time = time()
            try:
                ret, frame = player.read()
            except AssertionError:
                print("Video ended")
                break
            assert ret
            if i%1==0:
                results = self.score_frame(frame)
                frame = self.plot_boxes(results, frame)
                end_time = time()
                fps = 1/np.round(end_time - start_time, 3)
                print(f"Frames Per Second : {fps}")
            
            i+=1
            cv2.imshow('frame',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break


In [ ]:
# Create a new birds detection object and execute.
detect = BirdsDetection(source = 'rtsp://admin:@192.168.1.123/user=admin_password=_channel=1_stream=1.sdp',
                        conf_lvl=0.5,
                        )
detect()



/home/costia/anaconda3/envs/DL/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Founded: tensor([0.4883, 0.3215, 0.3192, 0.1900, 0.1113, 0.1029, 0.0964, 0.0897, 0.0889,
        0.0824, 0.0818, 0.0780, 0.0579, 0.0562, 0.0553, 0.0538, 0.0535, 0.0504],
       device='cuda:0')
Frames Per Second : 5.847953216374268
Founded: tensor([0.5001, 0.3095, 0.3078, 0.1881, 0.1053, 0.0994, 0.0935, 0.0914, 0.0891,
        0.0862, 0.0775, 0.0751, 0.0596, 0.0580, 0.0552, 0.0546, 0.0534, 0.0531],
       device='cuda:0')
Frames Per Second : 12.048192771084336
Founded: tensor([0.5103, 0.3184, 0.3138, 0.1823, 0.1088, 0.0985, 0.0959, 0.0922, 0.0916,
        0.0857, 0.0791, 0.0766, 0.0710, 0.0631, 0.0580, 0.0578, 0.0551, 0.0548],
       device='cuda:0')
Frames Per Second : 19.23076923076923
Founded: tensor([0.5249, 0.3104, 0.3096, 0.1929, 0.1069, 0.1013, 0.0968, 0.0961, 0.0938,
        0.0860, 0.0853, 0.0817, 0.0752, 0.0729, 0.0624, 0.0621, 0.0594, 0.0568,
        0.0531], device='cuda:0')
Frames Per Second : 20.0
Founded: tensor([0.5277, 0.3329, 0.3098, 0.2021, 0.1051, 0.1037, 0.1017, 0.

Founded: tensor([0.5824, 0.2637, 0.2593, 0.1982, 0.1684, 0.1583, 0.1295, 0.0858, 0.0821,
        0.0792, 0.0777, 0.0762, 0.0719, 0.0660, 0.0642, 0.0616, 0.0591, 0.0585,
        0.0574, 0.0549], device='cuda:0')
Frames Per Second : 22.72727272727273
Founded: tensor([0.5800, 0.2768, 0.2396, 0.1618, 0.1489, 0.1487, 0.1291, 0.1092, 0.0907,
        0.0803, 0.0789, 0.0769, 0.0729, 0.0706, 0.0672, 0.0665, 0.0582, 0.0569,
        0.0538, 0.0537], device='cuda:0')
Frames Per Second : 7.751937984496124
Founded: tensor([0.5817, 0.2619, 0.2363, 0.1820, 0.1600, 0.1577, 0.1265, 0.1115, 0.0883,
        0.0772, 0.0765, 0.0739, 0.0677, 0.0669, 0.0663, 0.0653, 0.0607, 0.0593,
        0.0550, 0.0541], device='cuda:0')
Frames Per Second : 5.847953216374268
Founded: tensor([0.5760, 0.2630, 0.2532, 0.1908, 0.1550, 0.1389, 0.1244, 0.1179, 0.0871,
        0.0796, 0.0794, 0.0776, 0.0759, 0.0667, 0.0637, 0.0626, 0.0558, 0.0556,
        0.0548, 0.0524], device='cuda:0')
Frames Per Second : 6.578947368421053
Foun

Founded: tensor([0.4542, 0.3158, 0.2693, 0.2611, 0.1727, 0.1382, 0.1366, 0.1272, 0.1006,
        0.0967, 0.0894, 0.0876, 0.0829, 0.0819, 0.0778, 0.0676, 0.0645, 0.0581,
        0.0548, 0.0529, 0.0528], device='cuda:0')
Frames Per Second : 5.58659217877095
Founded: tensor([0.4742, 0.3112, 0.2908, 0.2640, 0.2027, 0.1580, 0.1380, 0.1378, 0.0962,
        0.0919, 0.0912, 0.0878, 0.0817, 0.0776, 0.0774, 0.0689, 0.0549, 0.0548,
        0.0543, 0.0526, 0.0524, 0.0501], device='cuda:0')
Frames Per Second : 6.944444444444445
Founded: tensor([0.4612, 0.3092, 0.3009, 0.2741, 0.1477, 0.1417, 0.1394, 0.1026, 0.0949,
        0.0942, 0.0892, 0.0889, 0.0834, 0.0774, 0.0677, 0.0665, 0.0663, 0.0589,
        0.0584, 0.0542, 0.0535, 0.0527, 0.0507], device='cuda:0')
Frames Per Second : 4.6082949308755765
Founded: tensor([0.4686, 0.3129, 0.2992, 0.2822, 0.1537, 0.1408, 0.1382, 0.1033, 0.0982,
        0.0981, 0.0934, 0.0909, 0.0896, 0.0785, 0.0775, 0.0769, 0.0724, 0.0687,
        0.0582, 0.0570, 0.0551, 0.05

Founded: tensor([0.3484, 0.3264, 0.2198, 0.1770, 0.1571, 0.1221, 0.0928, 0.0850, 0.0759,
        0.0721, 0.0691, 0.0681, 0.0640, 0.0631, 0.0550, 0.0522, 0.0518, 0.0506],
       device='cuda:0')
Frames Per Second : 6.993006993006993
Founded: tensor([0.4585, 0.3327, 0.2504, 0.2386, 0.1454, 0.1231, 0.1185, 0.1120, 0.0973,
        0.0914, 0.0858, 0.0794, 0.0686, 0.0681, 0.0669, 0.0660, 0.0632, 0.0600,
        0.0599, 0.0557, 0.0545], device='cuda:0')
Frames Per Second : 6.849315068493151
Founded: tensor([0.4259, 0.3385, 0.2676, 0.2191, 0.1508, 0.1257, 0.1135, 0.1031, 0.1024,
        0.0965, 0.0801, 0.0793, 0.0695, 0.0692, 0.0653, 0.0619, 0.0605, 0.0593,
        0.0581, 0.0579], device='cuda:0')
Frames Per Second : 6.944444444444445
Founded: tensor([0.4755, 0.3492, 0.2812, 0.2344, 0.1639, 0.1222, 0.1116, 0.1047, 0.1030,
        0.0976, 0.0869, 0.0777, 0.0715, 0.0714, 0.0676, 0.0648, 0.0623, 0.0622,
        0.0584, 0.0543], device='cuda:0')
Frames Per Second : 5.6818181818181825
Founded: ten

...

Founded: tensor([0.4710, 0.2690, 0.2349, 0.1592, 0.1550, 0.1281, 0.1173, 0.1023, 0.0754,
        0.0740, 0.0725, 0.0706, 0.0674, 0.0653, 0.0564, 0.0513, 0.0510],
       device='cuda:0')
Frames Per Second : 5.0
Founded: tensor([0.4692, 0.2822, 0.2293, 0.1582, 0.1524, 0.1373, 0.1194, 0.1020, 0.0854,
        0.0743, 0.0735, 0.0680, 0.0661, 0.0634, 0.0562, 0.0521, 0.0519],
       device='cuda:0')
Frames Per Second : 7.142857142857142
Founded: tensor([0.4680, 0.2835, 0.2298, 0.1561, 0.1551, 0.1229, 0.1220, 0.1054, 0.0947,
        0.0816, 0.0755, 0.0656, 0.0642, 0.0630, 0.0545, 0.0526, 0.0516, 0.0501],
       device='cuda:0')
Frames Per Second : 5.9171597633136095
Founded: tensor([0.4649, 0.2841, 0.2224, 0.1551, 0.1437, 0.1212, 0.1032, 0.0787, 0.0783,
        0.0746, 0.0693, 0.0677, 0.0596, 0.0544, 0.0544], device='cuda:0')
Frames Per Second : 3.9215686274509802
Founded: tensor([0.4663, 0.2869, 0.2224, 0.1500, 0.1413, 0.1226, 0.1033, 0.0813, 0.0796,
        0.0762, 0.0734, 0.0692, 0.0669, 0.

Founded: tensor([0.4235, 0.2828, 0.2820, 0.1724, 0.1203, 0.1099, 0.1012, 0.0893, 0.0890,
        0.0867, 0.0810, 0.0798, 0.0699, 0.0658, 0.0657, 0.0564, 0.0560, 0.0554,
        0.0544, 0.0503], device='cuda:0')
Frames Per Second : 4.484304932735426
Founded: tensor([0.4245, 0.2789, 0.2749, 0.1816, 0.1203, 0.1098, 0.1059, 0.0917, 0.0908,
        0.0847, 0.0799, 0.0790, 0.0695, 0.0675, 0.0625, 0.0594, 0.0590, 0.0590,
        0.0569, 0.0503], device='cuda:0')
Frames Per Second : 7.142857142857142
Founded: tensor([0.4327, 0.2942, 0.2763, 0.1858, 0.1229, 0.1158, 0.0925, 0.0913, 0.0835,
        0.0757, 0.0722, 0.0691, 0.0667, 0.0661, 0.0630, 0.0627, 0.0615, 0.0590,
        0.0564, 0.0511], device='cuda:0')
Frames Per Second : 7.575757575757575
Founded: tensor([0.4329, 0.2897, 0.2701, 0.1816, 0.1190, 0.1166, 0.0924, 0.0924, 0.0797,
        0.0759, 0.0727, 0.0709, 0.0676, 0.0672, 0.0627, 0.0603, 0.0600, 0.0594,
        0.0566, 0.0504], device='cuda:0')
Frames Per Second : 5.025125628140703
Foun

Founded: tensor([0.4707, 0.3036, 0.2214, 0.1870, 0.1789, 0.1393, 0.1010, 0.0901, 0.0800,
        0.0703, 0.0666, 0.0661, 0.0655, 0.0646, 0.0570, 0.0549, 0.0538],
       device='cuda:0')
Frames Per Second : 6.172839506172839
Founded: tensor([0.4742, 0.3147, 0.2236, 0.1790, 0.1765, 0.1376, 0.1000, 0.0887, 0.0803,
        0.0689, 0.0670, 0.0662, 0.0653, 0.0647, 0.0601, 0.0584, 0.0543, 0.0502],
       device='cuda:0')
Frames Per Second : 6.289308176100628
Founded: tensor([0.4729, 0.3268, 0.2393, 0.1813, 0.1781, 0.1406, 0.1039, 0.1003, 0.0744,
        0.0706, 0.0687, 0.0680, 0.0668, 0.0649, 0.0597, 0.0593, 0.0560, 0.0520,
        0.0503], device='cuda:0')
Frames Per Second : 7.8125
Founded: tensor([0.4702, 0.3265, 0.2430, 0.1946, 0.1832, 0.1400, 0.1016, 0.0979, 0.0977,
        0.0742, 0.0715, 0.0713, 0.0691, 0.0675, 0.0650, 0.0600, 0.0600, 0.0574,
        0.0522, 0.0519, 0.0512], device='cuda:0')
Frames Per Second : 5.714285714285714
Founded: tensor([0.4776, 0.3259, 0.2387, 0.1920, 0.1868, 

Founded: tensor([0.5114, 0.3357, 0.2067, 0.1456, 0.1329, 0.1268, 0.0924, 0.0851, 0.0816,
        0.0808, 0.0720, 0.0711, 0.0667, 0.0654, 0.0631, 0.0612, 0.0608, 0.0553,
        0.0528], device='cuda:0')
Frames Per Second : 5.617977528089888
Founded: tensor([0.5162, 0.3442, 0.2257, 0.1553, 0.1474, 0.1146, 0.0941, 0.0874, 0.0811,
        0.0804, 0.0800, 0.0670, 0.0664, 0.0647, 0.0624, 0.0601, 0.0564, 0.0519,
        0.0511], device='cuda:0')
Frames Per Second : 5.88235294117647
Founded: tensor([0.5286, 0.3498, 0.2108, 0.1529, 0.1440, 0.1116, 0.0921, 0.0842, 0.0803,
        0.0779, 0.0764, 0.0764, 0.0680, 0.0673, 0.0656, 0.0640, 0.0554],
       device='cuda:0')
Frames Per Second : 5.74712643678161
Founded: tensor([0.5069, 0.3533, 0.2041, 0.1418, 0.1334, 0.0998, 0.0957, 0.0843, 0.0839,
        0.0812, 0.0790, 0.0768, 0.0756, 0.0694, 0.0667, 0.0663, 0.0511],
       device='cuda:0')
Frames Per Second : 6.8965517241379315
Founded: tensor([0.5120, 0.3752, 0.1862, 0.1474, 0.1196, 0.0922, 0.0885

Founded: tensor([0.5640, 0.3804, 0.2636, 0.1702, 0.1329, 0.1066, 0.1005, 0.0958, 0.0866,
        0.0783, 0.0765, 0.0719, 0.0697, 0.0628, 0.0540, 0.0513],
       device='cuda:0')
Frames Per Second : 7.633587786259541
Founded: tensor([0.5648, 0.3718, 0.2608, 0.1755, 0.1365, 0.0985, 0.0977, 0.0965, 0.0815,
        0.0756, 0.0750, 0.0719, 0.0682, 0.0612, 0.0521, 0.0511],
       device='cuda:0')
Frames Per Second : 6.5359477124183005
Founded: tensor([0.5590, 0.3618, 0.2589, 0.1712, 0.1346, 0.1014, 0.0975, 0.0950, 0.0816,
        0.0768, 0.0747, 0.0632, 0.0559, 0.0507], device='cuda:0')
Frames Per Second : 5.617977528089888
Founded: tensor([0.5435, 0.3649, 0.2707, 0.1828, 0.1299, 0.1007, 0.0999, 0.0986, 0.0792,
        0.0786, 0.0780, 0.0677, 0.0626, 0.0571], device='cuda:0')
Frames Per Second : 6.993006993006993
Founded: tensor([0.5488, 0.3656, 0.2619, 0.1891, 0.1439, 0.1010, 0.1004, 0.0991, 0.0799,
        0.0797, 0.0792, 0.0649, 0.0633, 0.0579], device='cuda:0')
Frames Per Second : 5.1282